In [1]:
import pandas as pd
import numpy as np

df_people = pd.read_csv('Traffic_Crashes_-_People_20251030.csv')

C:\Users\alova\AppData\Local\Temp\ipykernel_9464\2594960417.py:4: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_people = pd.read_csv('Traffic_Crashes_-_People_20251030.csv')


In [17]:
#Seleccionamos las columnas a usar
df_people_people = df_people[['PERSON_ID', 'PERSON_TYPE', 'CRASH_RECORD_ID', 'VEHICLE_ID', 'CRASH_DATE', 'SEX', 'AGE', 'SAFETY_EQUIPMENT', 'AIRBAG_DEPLOYED', 'INJURY_CLASSIFICATION']]
df_people_people.columns

Index(['PERSON_ID', 'PERSON_TYPE', 'CRASH_RECORD_ID', 'VEHICLE_ID',
       'CRASH_DATE', 'SEX', 'AGE', 'SAFETY_EQUIPMENT', 'AIRBAG_DEPLOYED',
       'INJURY_CLASSIFICATION'],
      dtype='object')

In [18]:
for c in df_people_people:
    print("------- Atributo:", c)
    print("Valores nulos: ", df_people_people[c].isna().sum())
    print("Valores distintos: ", df_people_people[c].unique())

------- Atributo: PERSON_ID
Valores nulos:  0
Valores distintos:  ['O2179723' 'P480828' 'P480829' ... 'O595730' 'O595731' 'P134373']
------- Atributo: PERSON_TYPE
Valores nulos:  0
Valores distintos:  ['DRIVER' 'PASSENGER' 'PEDESTRIAN' 'BICYCLE' 'NON-MOTOR VEHICLE'
 'NON-CONTACT VEHICLE']
------- Atributo: CRASH_RECORD_ID
Valores nulos:  0
Valores distintos:  ['62c4e58eb2cdf2c7c0d8693b02e2e65a5369f227d759977d6aaa75741f0795afa3e0ff6a46eab367944e1b65257318b82941889b3b9715f181fcb2b7d43deacf'
 '6da1bcd91641838b96db4ce7d075137a4d215e06452fee9fb28fa4233b42b61a9669fa8c38f0c0b7e630e72120d14f237020362b1d5440f2768721e7e64f1ae5'
 'e7d969ad79f62838ae1fdaf3d0866a6227966a68cce8be318fddb8d7d16789c80df0a9a40b030204374bf56dffb6c8e009d5b4b3cd79088d2a5f880a8f992c46'
 ...
 'f62e27317feb174811cf4fefeb9fa1064fea6c0619a8738e23545309256850d853008824fcf0ac07a71ef076b34be514e532e10d4a441c72c619b63e6193d72a'
 '19fb5af681f833c2af85734245f737fa6fbe62ac1ea379e14e22f71138add54f31f27960167f0ea70e7575704c5c7beb9a110fa

In [19]:
#verificar que no hay IDs duplicados
df_people_people['PERSON_ID'].duplicated().sum()


np.int64(0)

In [20]:
df_people_people.columns = (
    df_people_people.columns
      .astype(str)
      .str.replace("\ufeff", "", regex=False)       
      .str.replace(r"^[^A-Za-z0-9]+", "", regex=True) 
      .str.strip()
      .str.replace(r"\s+", "_", regex=True)         
)

In [32]:
#convertir los datos basura a None, para que en postgres se consideren NULL
df_people_people = df_people_people.replace(['', "", ' ', 'NA', 'N/A', 'null', 'NULL'], None)
df_people_people = df_people_people.where(df_people_people.notna(), None) 

In [33]:
df_people_people['VEHICLE_ID'] = (
    df_people_people['VEHICLE_ID']
    .astype(str)                      # convert everything to string safely
    .str.replace(r'\.0$', '', regex=True)  # remove trailing .0
)

# Convert to integers (nullable)
df_people_people['VEHICLE_ID'] = pd.to_numeric(df_people_people['VEHICLE_ID'], errors='coerce').astype('Int64')


In [34]:
print(df_people_people.columns)

Index(['PERSON_ID', 'PERSON_TYPE', 'CRASH_RECORD_ID', 'VEHICLE_ID',
       'CRASH_DATE', 'SEX', 'AGE', 'SAFETY_EQUIPMENT', 'AIRBAG_DEPLOYED',
       'INJURY_CLASSIFICATION'],
      dtype='object')


In [35]:
#la columna de AGE tiene algunos valores negativos
(df_people_people['AGE'] < 0).sum()

np.int64(0)

In [36]:
df_people_people.loc[df_people_people['AGE'] < 0, 'AGE'] = None
#verificamos que ya no haya edades negativas
(df_people_people['AGE'] < 0).sum()

np.int64(0)

In [37]:
#reemplazar los valores de 'USAGE UNKNOWN' del atributo SAFETY EQUIPMENT con None
df_people_people['SAFETY_EQUIPMENT'] = df_people_people['SAFETY_EQUIPMENT'].replace("USAGE UNKNOWN", None)
#comprobamos
(df_people_people['SAFETY_EQUIPMENT'] == "USAGE UNKNOWN").sum()

np.int64(0)

In [38]:
#reemplazar los valores de "DEPLOYMENT UNKNOWN" y "NOT APPLICABLE" con None
df_people_people['AIRBAG_DEPLOYED'] = df_people_people['AIRBAG_DEPLOYED'].replace("DEPLOYMENT UNKNOWN", None)
df_people_people['AIRBAG_DEPLOYED'] = df_people_people['AIRBAG_DEPLOYED'].replace("NOT APPLICABLE", None)

#comprobamos
print((df_people_people['AIRBAG_DEPLOYED'] == "DEPLOYMENT UNKNOWN").sum())
print((df_people_people['AIRBAG_DEPLOYED'] == "NOT APPLICABLE UNKNOWN").sum())

0
0


In [ ]:
#Cambiamos los tipos de datos 
df_people_people = df_people_people.assign(
    PERSON_ID = df_people_people['PERSON_ID'].astype('string'),
    PERSON_TYPE = df_people_people['PERSON_TYPE'].astype('string'),
    CRASH_RECORD_ID = df_people_people['CRASH_RECORD_ID'].astype('string'),
    VEHICLE_ID = pd.to_numeric(df_people_people['VEHICLE_ID'], errors='coerce').astype('Int64'),
    CRASH_DATE = pd.to_datetime(df_people_people['CRASH_DATE'], errors='coerce'),
    SEX = df_people_people['SEX'].astype('string'),
    AGE = df_people_people['AGE'].astype('Int64'),
    SAFETY_EQUIPMENT = df_people_people['SAFETY_EQUIPMENT'].astype('string'),
    AIRBAG_DEPLOYED = df_people_people['AIRBAG_DEPLOYED'].astype('string'),
    INJURY_CLASSIFICATION = df_people_people['INJURY_CLASSIFICATION'].astype('string')
)


In [41]:
df_people_people.to_csv("people_people.csv")